In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
!cd yolov5
!pip install -r ./yolov5/requirements.txt  # install

In [1]:
import random
import numpy as np
import pandas as pd
import cv2
from IPython.display import display
import matplotlib.pyplot as plt
import os
import ast
import shutil

In [2]:
train = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv", index_col="image_id")
train.head(10)

In [3]:
img = cv2.imread("../input/tensorflow-great-barrier-reef/train_images/video_0/978.jpg")
# from cv2 import cv2_imshow

start_pt = (825, 224)
end_pt = (875, 259)
color = (0, 0, 255)
thickness = 2
image = cv2.rectangle(img, start_pt, end_pt, color, thickness)
plt.imshow(image)
print(image.shape)

In [ ]:
!mkdir -p ./datasets/coral/labels
!mkdir -p ./datasets/coral/images

path_to_txt_folder = "./datasets/coral/labels"
final_path_to_images = "./datasets/coral/images"

def txt_data(path):
    for folder in os.listdir(path):
        path_to_folder = os.path.join(path, folder)
        _, video_id = folder.split("_")

        for image in os.listdir(path_to_folder):
            path_to_image = os.path.join(path_to_folder, image)

            image_id, img_ext = image.split(".")
            image_match = video_id + "-" + image_id

            img_temp = cv2.imread(path_to_image)
            height, width, channels = img_temp.shape

            image_data = train.loc[image_match]
            annotation_list = ast.literal_eval(image_data.annotations)

            if bool(annotation_list) == True:
                path_to_file = os.path.join(path_to_txt_folder, image_match + ".txt")
                file_ = open(path_to_file, "w")
                for i in annotation_list:
                    x_min = i['x']
                    y_min = i['y']
                    box_width = i['width']
                    box_height = i['height']
                    class_label=str(0)
                    x_min = str(round((x_min+(box_width/2))/width, 10))
                    y_min = str(round((y_min+(box_height/2))/height, 10))
                    box_width = str(round(box_width/width, 10))
                    box_height = str(round(box_height/height, 10))
                    file_.write(class_label+" "+x_min+" "+y_min+" "+box_width+" "+" "+box_height)
                file_.close()
    
                image_new_name = os.path.join(final_path_to_images, image_match + "." + img_ext)
#             new_path_to_image = os.path.join(final_path_to_images, image_new_name)
                shutil.copyfile(path_to_image, image_new_name)

txt_data("../input/tensorflow-great-barrier-reef/train_images")

In [ ]:
img = cv2.imread("./datasets/coral/images/0-978.jpg")
# from google.colab.patches import cv2_imshow

start_pt = (825, 224)
end_pt = (875, 259)
color = (0, 0, 255)
thickness = 2
image = cv2.rectangle(img, start_pt, end_pt, color, thickness)
plt.imshow(image)
print(image.shape)

In [ ]:
shutil.copyfile("../input/coralyaml/coral.yaml", "./yolov5/data/coral.yaml")
shutil.copyfile("../input/coralyaml/hyp.yaml", "./yolov5/data/hyps/hyp.yaml")

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_key")

!pip install wandb
!wandb login $secret_value_0

In [ ]:
## Train YOLOv5m on coral_dataset for 3 epochs
#!python ./yolov5/train.py --img 1280 --rect --batch 8 --epochs 15 --data ./yolov5/data/coral.yaml --hyp ./yolov5/data/hyps/hyp.yaml --weights yolov5m.pt

In [4]:
import torch
ckpt_path = "../input/easy-firefly-weights/best_yolov5s_0.78_precision_logical_oath_36.pt"

model = torch.hub.load('./yolov5', 'custom',
                           path=ckpt_path,
                           source='local',
                       force_reload=True)

model.conf = 0.01
model.iou = 0.25

#results = model("./datasets/coral/images/0-4600.jpg")
#results.print()
#results.pandas().xyxy[0]

In [5]:
results = model("../input/tensorflow-great-barrier-reef/train_images/video_1/100.jpg")
#results.print()
results_df=results.pandas().xyxy[0]
results_df

In [7]:
def result_string_calculator(results_dataframe):
    if results_dataframe.shape[0] != 0:
        for index,row in results_dataframe.iterrows():
            x_center = row['xmin'] + ((row['xmax'] - row['xmin'])/2)
            y_center = row['ymin'] + ((row['ymax'] - row['ymin'])/2)
            height= row['ymax'] - row['ymin']
            width = row['xmax'] - row['xmin']
            confidence_score = row['confidence']
        output_string = "{} {} {} {} {}".format(confidence_score,round(x_center),round(y_center),round(width),round(height))
    else:
        output_string = "0 0 0 0 0"
    return output_string

In [8]:
result_string_calculator(results_df)

In [9]:
import greatbarrierreef
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (pixel_array, sample_prediction_df) in iter_test:
    results = model(pixel_array)
    results_df=results.pandas().xyxy[0]
    sample_prediction_df['annotations'] =  result_string_calculator(results_df) # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions